<img src='https://user-images.githubusercontent.com/6457691/90080969-0f758d00-dd47-11ea-8191-fa12fd2054a7.png' width = '200' align = 'right'>

## *AIB / SECTION 4 / SPRINT 1 / NOTE 3*

---

# N413. 더 나은 신경망 학습을 위한 방법들 - 학습률(Learning rate), 가중치 초기화(Weight Initialization) & Regularization

## 더 좋은 결과를 만들어봅시다!

지난 강의를 통해 신경망의 기본 구조가 되는 퍼셉트론과 신경망 학습에 대해 이해하고,<br/>
Keras를 사용하여 기본적인 신경망을 구축해 보았습니다.

이번 강의에서는 신경망 학습이 더 잘되도록 하는 방법에 대해 알아보도록 하겠습니다.

#### 1. 학습률 감소/계획법(Learning rate Decay/Scheduling)

첫 번째로 알아볼 기법은 **학습률 감소 및 계획법(Learning rate Decay/Scheduling)** 입니다.<br/>
위 방법에 대해 알아보기 전에 먼저 학습률에 대해 알아보도록 하겠습니다.

- **학습률(Learning rate)**

**<font color="ff6f61">학습률(Learning rate, `lr`)</font>**이란 매 가중치에 대해 구해진 기울기 값을 얼마나 경사 하강법에 적용할 지를 결정하는 하이퍼파라미터입니다.<br/>
지난 시간에 보았던 경사 하강법 수식에서 학습률이 어디에 위치하는지 확인해봅시다.

<img src="https://i.imgur.com/ic91umJ.png" height="200"/>

위 식에서 볼 수 있는 것처럼 해당 지점에서의 기울기를 구하여 기울기가 감소하는$(-)$ 방향으로 이동하게 되는데요.<br/>
학습률은 **얼마나 이동할 지를 조정하는 하이퍼파라미터**입니다.

경사 하강법이 산길을 내려가는 과정이라면 학습률은 **보폭을 결정**하게 됩니다.<br/>
학습률이 크면 보폭이 크니 Iteration 마다 성큼성큼 이동하고, 작으면 보폭이 작아 조금씩만 이동하게 됩니다.<br/>
그렇다면 학습률을 잘못 설정하면 어떻게 될까요?

- **학습률이 너무 크거나 작으면 어떻게 될까요?**

> ❗️ ***아래는 학습률이 너무 클 때와 작을 때의 경사하강법을 나타낸 그림입니다.<br/>
그림을 기억하면서 최적의 학습률이 왜 중요한 지에 대해 생각해봅시다.***

<img src="https://i.imgur.com/RfBFgKs.png" height="300">

그림에서 확인할 수 있는 것처럼<br/>
**학습률이 너무 낮으면 최적점에 이르기까지 너무 오래 걸리거나, 주어진 Iteration 내에서 최적점에 도달하는 데 실패**하기도 합니다.<br/>
반대로 **너무 높으면 경사 하강 과정에서 발산하면서 모델이 최적값을 찾을 수 없게** 되어버립니다.

그렇기 때문에 최적의 학습률을 찾는 것은 학습에서 중요한 요소인데요.<br/>
위와 같은 문제를 해결하기 위해서 사용되는 방법이 **학습률 감소/계획법**입니다.


- **학습률 감소/계획법(Learning rate Decay/Scheduling)**
<br/>

1. **학습률 감소(Learning rate Decay)**

    학습률 감소는 Adagrad, RMSprop, Adam 과 같은 주요 옵티마이저에 이미 구현되어 있기 때문에 쉽게 적용할 수 있습니다.<br/>
    해당 옵티마티저의 하이퍼파라미터를 조정하면 감소 정도를 변화시킬 수 있습니다.

    아래와 같이 **`.compile`** 내에 있는 **`optimizer=`** 에 Adam 등의 옵티마이저 적용 후<br/> 내부 하이퍼파라미터를 변경하면 학습률 감소를 적용할 수 있습니다.

```python
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1 = 0.89)
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])
```

2. **학습률 계획법(Learning rate Scheduling)**<br/>

    아래 그래프와 같이 Warm-up Step 을 포함한 학습률 계획 방법을 적용하기도 합니다.
    

    

<img src="https://i.imgur.com/Ida3BHl.png" height="200"/>


학습률 계획은 다음과 같이 **`.experimental`** 내부의 함수를 사용하여 설계할 수 있습니다.

```python
first_decay_steps = 1000
initial_learning_rate = 0.01
lr_decayed_fn = (
  tf.keras.experimental.CosineDecayRestarts(
      initial_learning_rate,
      first_decay_steps))
```

```python
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_decayed_fn)
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])
```

#### 2. 가중치 초기화(Weight Initialization)

초기 가중치 설정과 관련된 **<font color="ff6f61">가중치 초기화(Weight initialization)</font>**는 신경망에서 매우 중요한 요소입니다.

아래에서는 먼저 가중치를 정규분포로 초기화하였을 때의 문제를 알아보고 <br/>
흔히 사용되는 **Xavier 초기화**와 **He 초기화**에 대해 알아보도록 하겠습니다.

> ❗️ ***아래 가중치 초기화 방법의 수식을 외울 필요는 없습니다.<br/>
일단은 어떤 초기화 방법이 있고 해당 초기화 방법이 언제 사용되는지만 기억하고 넘어갑시다.***<br/>
❗️ ***실제로는 더 많은 초기화 방법이 있습니다. 모든 초기화 방법을 기억할 필요는 없습니다.***

1. **표준편차를 1인 정규분포로 가중치를 초기화 할 때 각 층의 활성화 값 분포**

표준편차가 일정한 정규분포로 가중치를 초기화 해 줄 때에는 대부분의 활성화 값이 0과 1에 위치하는 것을 볼 수 있습니다.<br/>
활성값이 고르지 못할 경우 학습이 제대로 이루어지지 않는데요.<br/>
그렇기 때문에 가장 간단한 방법임에도 잘 사용되지 않습니다.

<img src="https://i.imgur.com/Zd6OsJT.png" height=300/>

2. **Xavier 초기화를 해주었을 때의 활성화 값의 분포**

**Xavier 초기화(Xavier initialization)**는 가중치를 표준편차가 고정값인 정규분포로 초기화 했을 때의 문제점을 해결하기 위해 등장한 방법입니다.<br/>
Xavier 초기화는 이전 층의 노드가 $n$ 개일 때, 현재 층의 가중치를 표준편차가 $\frac{1}{\sqrt{n}}$ 인 정규분포로 초기화합니다.<br/>
*(케라스에서 Xavier 초기화는 이전 층의 노드가 $n$ 개이고 현재 층의 노드가 $m$ 개일 때, 현재 층의 가중치를 표준편차가 $\frac{2}{\sqrt{n+m}}$ 인 정규분포로 초기화합니다.)*

<img src="https://i.imgur.com/NXziIT2.png" height="300"/>


3. **He 초기화를 해주었을 때의 활성화 값의 분포**

Xavier 초기화는 활성화 함수가 시그모이드(`Sigmoid`)인 신경망에서는 잘 동작합니다.<br/>
하지만 활성화 함수가 ReLU 일 경우에는 층이 지날수록 활성값이 고르지 못하게 되는 문제를 보이는데요.

이런 문제를 해결하기 위해 등장한 것이 바로 **He 초기화(He initialization)** 입니다.<br/>
He 초기화는 이전 층의 노드가 $n$ 개일 때, 현재 층의 가중치를 표준편차가 $\frac{2}{\sqrt{n}}$ 인 정규분포로 초기화합니다.<br/>
He 초기화를 적용하면 아래 그림처럼 층이 지나도 활성값이 고르게 유지되는 것을 확인할 수 있습니다.

<img src="https://i.imgur.com/gZRdLiM.png" height="300"/>

가중치 초기화 방법을 요약하면 다음과 같습니다.<br/>

> **Activation function에 따른 초기값 추천**
1. Sigmoid  ⇒  Xavier 초기화를 사용하는 것이 유리 
2. ReLU  ⇒  He 초기화 사용하는 것이 유리

가중치 초기화 방법은 이외에도 여러 가지가 있습니다.<br/>
케라스에서는 아래와 같은 가중치 초기화 방법을 제공하고 있습니다.

> ❗️ ***모든 가중치 초기화 방법을 알 필요는 없습니다.***

```python
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
```


## 머신러닝으로서의 딥러닝

아래와 비슷한 그림을 보신 적이 있으신가요?

<img src="https://i.imgur.com/1MMrBS5.png" height = "200"/>


위 그림은 **인공지능(Artificial Intelligence)**과 **머신러닝(Machine Learning)** 및 **딥러닝(Deep Learning)**의 관계를 나타내고 있습니다.

그림에서 볼 수 있는 것처럼 Section 4 에서 다루고 있는 딥러닝, 즉 인공 신경망도 역시 머신러닝 모델이라고 할 수 있는데요.<br/>
그렇기 때문에 머신러닝 모델에서 발생할 수 있는 문제가 동일하게 발생하고, 문제를 해결할 수 있는 방법 역시 동일하게 적용할 수 있습니다.


### 과적합(Overfitting)



인공 신경망의 노드 수와 층을 늘리다 보면 매개 변수가 상당히 많아집니다.

Fashion MNIST 예제를 풀기 위해서 구축한 신경망에서는 은닉층 없이 출력층만 설계했음에도 7,850개의 파라미터가 있었는데요.<br/>
딥러닝, 즉 은닉층이 3개 이상인 신경망에는 훨씬 더 많은 수의 파라미터가 있습니다.

머신러닝에서는 모델이 복잡해지면 **<font color="ff6f61">과적합(Overfitting)</font>** 문제가 발생하는 경향이 있는데요.<br/>
이번 강의에서는 과적합 방지를 위해 적용하는 방법에 대해서 알아보겠습니다. 


### 과적합 방지를 위한 방법들

#### 1. Weight Decay (가중치 감소)

첫 번째로 알아볼 방법은 **<font color="ff6f61">가중치 감소(Weight Decay)</font>** 입니다.

과적합은 가중치의 값이 클 때 주로 발생하는데요.<br/>
가중치 감소에서는 가중치가 너무 커지지 않도록 **가중치 값이 너무 커지지 않도록 조건을 추가**합니다.<br/>
이 과정에서 **손실 함수(Cost function)에 가중치와 관련된 항을 추가**됩니다.<br/>
조건을 어떻게 적용할지에 따라 L1 Regularization(LASSO), L2 Regularization(Ridge) 으로 나뉩니다.

각 방법에 따른 손실 함수 식은 다음과 같습니다.

$$
\begin{aligned}
L_1(\theta_w) &= \frac{1}{2} \sum_i (output_i - target_i)^2 + \color{blue}{\lambda} \cdot \color{red}{\Vert \theta_w \Vert_1}\\
L_2(\theta_w) &= \frac{1}{2} \sum_i (output_i - target_i)^2 + \color{blue}{\lambda} \cdot \color{red}{\Vert \theta_w \Vert_2}
\end{aligned}
$$

Keras 에서는 아래와 같이 가중치 감소를 적용하고 싶은 층에 `regularizer` 파라미터를 추가하면 됩니다.

> ❗️ ***`kernel_regularizer` 와 `activity_regularizer` 의 차이에 대해서는 일단 넘어갑시다.***

```python
Dense(64,
      kernel_regularizer=regularizers.l2(0.01),
      activity_regularizer=regularizers.l1(0.01))
```

#### 2. Dropout (드롭아웃)

**Dropout(드롭아웃)**은 Iteration 마다 **레이어 노드 중 일부를 사용하지 않으면서 학습을 진행하는 방법**입니다.<br/>
매 번 다른 노드가 학습되면서 전체 가중치가 과적합되는 것을 방지할 수 있습니다.<br/>
아래는 Dropout을 적용했을 때의 신경망에서 학습되는 노드를 나타낸 그림입니다.

<img src="https://i.imgur.com/rAyIZxV.png" height="300">

Dropout 을 적용할 때에는 0~1 사이의 실수를 입력하게 되는데요.<br/>
모델 내에 있는 특정 레이어의 노드 연결을 지정해 준 비율만큼 강제로 끊습니다.<br/>
매 Iteration 마다 랜덤하게 노드를 차단하여 다른 가중치를 학습하도록 조정하기 때문에 과적합을 방지할 수 있게 됩니다.

Keras 에서는 아래와 같이 Dropout 을 적용하고 싶은 층 다음에 `Dropout` 함수를 추가하면 됩니다.

```python
Dense(64,
      kernel_regularizer=regularizers.l2(0.01),
      activity_regularizer=regularizers.l1(0.01))
Dropout(0.5)
```

#### 3. Early Stopping (조기 종료)


첫 번째 방법은 **<font color="ff6f61">조기 종료(Early Stopping)</font>**입니다.<br/>
아래 그림에서 볼 수 있는 것처럼 **학습(Train) 데이터에 대한 손실은 계속 줄어들지만<br/>
검증(Validation) 데이터셋에 대한 손실은 증가한다면 학습을 종료**하도록 설정하는 방법입니다.

<img src="https://i.imgur.com/gTdMIuD.png" height="300">

이제 Fashion MNIST 예제에서 구축 신경망에 **조기 종료(Early Stopping)**를 적용하여 보겠습니다.

### Fashion MNIST 신경망 예제에 기법 적용하기

In [2]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import regularizers

import os
import numpy as np
import tensorflow as tf
import keras

In [3]:
np.random.seed(42)
tf.random.set_seed(42)

1. **데이터셋을 불러옵니다.**

In [4]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
print(X_train.shape, X_test.shape)

4431872/4422102 [==============================] - 0s 0us/step
(60000, 28, 28) (10000, 28, 28)


2. **데이터를 정규화(Normalization)합니다.**

In [5]:
X_train = X_train / 255.
X_test = X_test / 255.

3. **레이블의 개수와 형태를 확인합니다.**

In [6]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

4. **신경망 모델을 구축하고 Compile 합니다.**

구축 과정에서 위에서 학습하였던 **Weight Decay(가중치 감소), Dropout(드롭아웃)**을 적용해봅시다.

In [7]:
# 기본적인 신경망을 만드는 코드
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64,
          kernel_regularizer=regularizers.l2(0.01),
          activity_regularizer=regularizers.l1(0.01)),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

Compile 설정에서 위에서 학습하였던 **Learning rate Decay(학습률 감소)**를 적용해봅니다.

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1 = 0.89)
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 64)                50240     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________


5. **신경망 모델을 학습합니다.**

먼저 학습 과정에서 **Early Stopping(조기 종료)**를 적용할 수 있도록<br/>
파라미터 저장 경로와 조기 종료 옵션을 설정하여 줍니다.

In [10]:
# 파라미터 저장 경로를 설정하는 코드입니다.
checkpoint_filepath = "FMbest.hdf5"

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

In [11]:
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None)

In [12]:
model.fit(X_train, y_train, batch_size=32, epochs=30, verbose=1, 
          validation_data=(X_test,y_test), 
          callbacks=[early_stop, save_best])

Epoch 1/30
1855/1875 [============================>.] - ETA: 0s - loss: 1.2151 - accuracy: 0.7680
Epoch 00001: val_loss improved from inf to 0.86058, saving model to FMbest.hdf5
1875/1875 [==============================] - 8s 3ms/step - loss: 1.2122 - accuracy: 0.7682 - val_loss: 0.8606 - val_accuracy: 0.8136
Epoch 2/30
1857/1875 [============================>.] - ETA: 0s - loss: 0.9307 - accuracy: 0.7883
Epoch 00002: val_loss did not improve from 0.86058
1875/1875 [==============================] - 4s 2ms/step - loss: 0.9306 - accuracy: 0.7884 - val_loss: 0.8889 - val_accuracy: 0.7957
Epoch 3/30
1853/1875 [============================>.] - ETA: 0s - loss: 0.9067 - accuracy: 0.7920
Epoch 00003: val_loss improved from 0.86058 to 0.85553, saving model to FMbest.hdf5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.9084 - accuracy: 0.7915 - val_loss: 0.8555 - val_accuracy: 0.7969
Epoch 4/30
1868/1875 [============================>.] - ETA: 0s - loss: 0.8966 - accuracy: 0

6. **조기종료 직전의 모델을 사용하여 평가를 진행합니다.**

In [13]:
model.predict(X_test[0:1])

array([[2.2075588e-05, 1.4129513e-06, 4.8530714e-05, 4.6155776e-05,
        8.1829887e-05, 1.5290031e-01, 8.9300127e-05, 2.5638691e-01,
        3.0169061e-03, 5.8740658e-01]], dtype=float32)

In [14]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.8547 - accuracy: 0.8055 - 504ms/epoch - 2ms/step


7. **콜백(Callback)에 의해 Best 모델의 파라미터가 제대로 저장되었는지 확인하고 해당 모델로 평가를 진행합니다.**

In [15]:
!ls 

FMbest.hdf5  sample_data


저장된 경로로부터 모델의 파라미터(가중치)를 불러옵니다.

In [16]:
model.load_weights(checkpoint_filepath)

불러온 모델을 사용하여 평가를 수행합니다.

In [17]:
model.predict(X_test[0:1])

array([[3.3812994e-05, 4.7637095e-05, 6.7187146e-05, 1.3367164e-04,
        5.9769911e-05, 2.1045013e-01, 8.0051337e-05, 1.3290186e-01,
        2.1713288e-03, 6.5405452e-01]], dtype=float32)

In [18]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)

313/313 [==============================] - 1s 2ms/step - loss: 0.7895 - accuracy: 0.8196
